In [ ]:
import pandas as pd
import mlflow
import json
import os
import socket
import time
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

from apps.ml.features import SensorFeatureTransformer

# MLflow setup (inside docker network use service name)
DEFAULT_TRACKING_URI = "http://mlflow:5000"
tracking_uri = os.getenv("MLFLOW_TRACKING_URI", DEFAULT_TRACKING_URI)
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Anomaly Detection")

def wait_for_port(host: str, port: int, timeout: int = 60, interval: float = 2.0):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=2):
                return True
        except OSError:
            time.sleep(interval)
    raise RuntimeError(f"MLflow not reachable at {host}:{port} after {timeout}s")

# Only wait if using the default internal URI
if "mlflow:5000" in tracking_uri:
    wait_for_port("mlflow", 5000)

print(f"MLflow tracking URI set to: {mlflow.get_tracking_uri()}")

In [ ]:
df = pd.read_csv('data/sensor_data.csv', parse_dates=['timestamp'])

# Initialize the ENHANCED feature transformer
feature_transformer = SensorFeatureTransformer(n_lags=5, scale_columns=['value', 'quality'])
X_transformed = feature_transformer.fit_transform(df)
feature_names = feature_transformer.get_feature_names_out()

print("Data transformed successfully with new transformer.")
print(f"Features ({len(feature_names)}): {feature_names}")
X_transformed.head()

In [ ]:
with mlflow.start_run(run_name="IsolationForest_v2_refined") as run:
    mlflow.log_param("model_type", "IsolationForest")
    mlflow.log_param("feature_engineering_version", "v2")

    # --- Log feature details for reproducibility ---
    mlflow.log_param("feature_count", len(feature_names))
    # Log feature names as a text artifact
    with open("feature_names.txt", "w") as f:
        json.dump(feature_names, f)
    mlflow.log_artifact("feature_names.txt")

    # --- Train Model ---
    contamination = 0.05
    mlflow.log_param("contamination", contamination)
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(X_transformed)

    # --- Log Metrics ---
    df['anomaly'] = model.predict(X_transformed)
    anomaly_rate = (df['anomaly'] == -1).mean()
    mlflow.log_metric("anomaly_rate", anomaly_rate)
    
    # Log feature statistics as a dictionary
    feature_stats = X_transformed.describe().to_dict()
    mlflow.log_dict(feature_stats, "feature_summary_stats.json")

    # --- Log Model and Artifacts ---
    mlflow.sklearn.log_model(model, "model", registered_model_name="anomaly_detector_refined_v2")

    plt.figure(figsize=(15, 7))
    sns.scatterplot(data=df, x='timestamp', y='value', hue='anomaly', palette={1: 'blue', -1: 'red'}, s=10)
    plt.title('Anomaly Detection (v2 - Refined Features)')
    plot_path = "docs/ml/anomaly_scatter_v2.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path, "plots")

    print(f"Run {run.info.run_name} complete. Anomaly rate: {anomaly_rate:.2%}")
    print(f"Model registered as 'anomaly_detector_refined_v2'. Check MLflow UI.")